In [1]:
from openai import OpenAI
import random
import json
from datetime import datetime

client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

# ------------------------------------------------------------
# PART 1. COMPLAINT COLLECTION (NO AI)
# ------------------------------------------------------------
print("\n" + "=" * 60)
print("HELLO! I'M LYUDMILA, YOUR MEDICAL ASSISTANT")
print("=" * 60)
print("\nI'll help gather information about how you're feeling")
print("before your appointment with the doctor. Everything you tell me")
print("will be passed on to your physician.")
print("-" * 60)

# Introduction
name = input("\n👤 What's your name? ")
print(f"\n🤖 Lyudmila: Nice to meet you, {name}!")

# Complaint collection
print("\n🤖 Lyudmila: Please tell me what's bothering you?")

complaints = []
questions = [
    "Anything else?",
    "Something else?",
    "What else is bothering you?",
    "I'm listening carefully, please continue. What else?"
]

while True:
    complaint = input("👤 You: ").strip()
    
    # Check for completion
    if any(phrase in complaint.lower() for phrase in 
           ["that's all", "no", "nothing else", "that's the main thing", "no more", "i think that's all", "all", "enough"]):
        break
    
    if complaint:  # if not empty
        complaints.append(complaint)
        
        # Different follow-up questions
        next_q = random.choice(questions)
        print(f"\n🤖 Lyudmila: {next_q}")

# Summarize
if complaints:    
    points = "\n".join([f"   • {complaint}" for complaint in complaints])
    summary = f"Let me make sure I understood correctly. You're concerned about:\n{points}"  
else:
    summary = "Let me make sure I understood correctly. Nothing is bothering you at the moment?"

print(f"\n🤖 Lyudmila: {summary}")
confirmation = input("👤 You: ")

# Corrections if needed
corrections = []
if "no" in confirmation.lower() or "not" in confirmation.lower():
    print("\n🤖 Lyudmila: Please correct me. What did I get wrong?")
    correction = input("👤 You: ")
    corrections.append(correction)
    # Additional clarification logic could be added here

# ------------------------------------------------------------
# PART 2. DETAILED ASSESSMENT (AI) - UNIFIED DIALOGUE
# ------------------------------------------------------------
print("\n" + "=" * 60)
print("NOW I'LL ASK A FEW DETAILED QUESTIONS")
print("=" * 60)

# Build prompt using points
if complaints:
    points = "\n".join([f"   • {complaint}" for complaint in complaints])
    
    system_prompt = f"""You are an empathetic medical assistant named Lyudmila. 
Patient {name} has confirmed the following complaints:

{points}

YOUR TASK:
Conduct a detailed assessment of EACH complaint in this list, like an experienced doctor would.

CONVERSATION RULES:
1. Start with the first complaint and work through them sequentially
2. For each complaint, find out:
   - When did it start?
   - How often does it occur?
   - What triggers/worsens it?
   - What alleviates it?
   - Have they tried any self-treatment?
3. Only move to the next complaint after getting answers to these questions
4. If the patient mentions a new issue not in the list, gently return to the list

⚠️ CRITICAL: Do NOT end the conversation until you've covered ALL complaints!

🚫 EARLY TERMINATION CONDITIONS (ONLY IF PATIENT SAYS EXACT PHRASES):
Patient must explicitly say one of these:
- "i'm tired of talking"
- "let's finish"
- "enough"
- "tell the doctor"
- "another time"
- "let's stop"
- "end this"

✅ IF THE PATIENT DOESN'T SAY THESE PHRASES - CONTINUE QUESTIONING! Even if they say "i'm tired" (without "of talking") - this could be a complaint about fatigue, not a request to stop.

At the end of the conversation (after all complaints or early termination), say: "Thank you, I've gathered all the necessary information and will pass it to your doctor." """

    # Properly initialize messages with alternating roles
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Hello! I have some concerns. Let's begin."},
        {"role": "assistant", "content": f"Let's go through your complaints in detail. Tell me about the first one: {complaints[0]}. When did it start?"}
    ]
    
    print(f"\n🤖 Lyudmila: Let's go through your complaints in detail. Tell me about the first one: {complaints[0]}. When did it start?")
    
    # Collect full dialogue transcript
    full_dialogue = [f"🤖 Lyudmila: Let's go through your complaints in detail. Tell me about the first one: {complaints[0]}. When did it start?"]
else:
    messages = [
        {"role": "system", "content": f"You are an empathetic medical assistant named Lyudmila. Patient {name} has reported no complaints. Thank them for their time and say that if any concerns arise, they can always return. Don't ask additional questions."},
        {"role": "user", "content": "I have no complaints."},
        {"role": "assistant", "content": f"Nice to meet you, {name}! If you have any questions or concerns later, please come back. Take care!"}
    ]
    print(f"\n🤖 Lyudmila: Nice to meet you, {name}! If you have any questions or concerns later, please come back. Take care!")
    full_dialogue = [f"🤖 Lyudmila: Nice to meet you, {name}! If you have any questions or concerns later, please come back. Take care!"]

# Main dialogue loop
patient_wants_to_stop = False

while complaints and not patient_wants_to_stop:
    user_input = input("👤 You: ")
    full_dialogue.append(f"👤 You: {user_input}")
    
    # EXPANDED EXIT CHECK (only exact phrases)
    exit_phrases = [
        "i'm tired of talking",
        "let's finish", 
        "enough questions",
        "tell the doctor",
        "another time", 
        "let's stop",
        "end this",
        "that's enough"
    ]
    
    # Check for exact or very close match
    should_exit = False
    user_lower = user_input.lower()
    for phrase in exit_phrases:
        if phrase in user_lower or user_lower in phrase:
            should_exit = True
            break
    
    if should_exit:
        patient_wants_to_stop = True
        # Add user message
        messages.append({"role": "user", "content": user_input})
        
        response = client.chat.completions.create(
            model="local-model",
            messages=messages,
            temperature=0.7
        )
        
        final_response = response.choices[0].message.content
        messages.append({"role": "assistant", "content": final_response})
        full_dialogue.append(f"🤖 Lyudmila: {final_response}")
        print(f"\n🤖 Lyudmila: {final_response}")
        break
    
    # Add user response
    messages.append({"role": "user", "content": user_input})
    
    response = client.chat.completions.create(
        model="local-model",
        messages=messages,
        temperature=0.7
    )
    
    next_response = response.choices[0].message.content
    # Add assistant response
    messages.append({"role": "assistant", "content": next_response})
    full_dialogue.append(f"🤖 Lyudmila: {next_response}")
    print(f"\n🤖 Lyudmila: {next_response}")
    
    # Check if model ended the conversation itself
    if "pass it to your doctor" in next_response.lower() and "gathered all the necessary information" in next_response.lower():
        break

# ------------------------------------------------------------
# PART 3. GENERATE SUMMARY FOR DOCTOR
# ------------------------------------------------------------
if complaints and full_dialogue:
    # Create prompt for summary generation
    summary_prompt = f"""You are medical assistant Lyudmila. Based on the conversation transcript with patient {name}, create a CONCISE STRUCTURED SUMMARY for the doctor.

CONVERSATION TRANSCRIPT:
{chr(10).join(full_dialogue)}

YOUR TASK:
Create a summary in the following format:

COMPLAINT 1: [brief title]
   • Onset: [when it started]
   • Characteristics: [how it manifests, frequency]
   • Triggers: [what worsens it]
   • Relieving factors: [what helps]
   • Treatment attempts: [what they tried, effect]

COMPLAINT 2: [brief title]
   • Onset: ...
   ... and so on for all complaints

ADDITIONAL:
   • Lifestyle/context: [important details about work, stress, habits]
   • Special notes: [what the doctor should know]

RULES:
- Only facts from the conversation, no speculation
- As concise as possible but informative
- If information for any point is missing - skip it
- Don't use markdown, only plain text"""

    summary_messages = [
        {"role": "system", "content": "You are a medical assistant creating a brief conversation summary for the doctor."},
        {"role": "user", "content": summary_prompt}
    ]
    
    try:
        summary_response = client.chat.completions.create(
            model="local-model",
            messages=summary_messages,
            temperature=0.3  # Low temperature for factual responses
        )
        medical_summary = summary_response.choices[0].message.content
    except Exception as e:
        medical_summary = "Could not generate summary automatically."

# ------------------------------------------------------------
# CLOSING (if model didn't say goodbye)
# ------------------------------------------------------------
if complaints and not patient_wants_to_stop and not any("pass it to your doctor" in msg.lower() for msg in full_dialogue[-3:]):
    print("\n" + "=" * 60)
    print(f"🤖 Lyudmila: Thank you for the conversation, {name}! I'll pass all the information to your doctor.")
    print("🤖 Lyudmila: See you at your appointment!")
    print("=" * 60)

# ------------------------------------------------------------
# SAVE REPORT FOR DOCTOR
# ------------------------------------------------------------
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M")
report = f"""
MEDICAL REPORT
================================
Patient: {name}
Date: {timestamp}

COLLECTED COMPLAINTS:
{'-' * 40}
"""
for i, complaint in enumerate(complaints, 1):
    report += f"{i}. {complaint}\n"

if corrections:
    report += f"\nPATIENT CORRECTIONS:\n"
    for corr in corrections:
        report += f"   • {corr}\n"

# Add AI summary for doctor
if complaints and full_dialogue:
    report += f"""

📋 AI-GENERATED SUMMARY FOR DOCTOR
{'-' * 40}
{medical_summary}
"""

report += f"""

📜 FULL CONVERSATION TRANSCRIPT:
{'-' * 40}
"""
for line in full_dialogue:
    report += line + "\n"

report += """
================================
Generated by AI Assistant Lyudmila
Requires doctor's review during appointment
"""

# Save to file
filename = f"report_{name}_{datetime.now().strftime('%Y%m%d_%H%M')}.txt"
with open(filename, "w", encoding="utf-8") as f:
    f.write(report)

print(f"\n✅ Report saved to file: {filename}")
print("\n" + "=" * 60)
print("Report includes:")
print("• Concise summary for quick review")
print("• Full transcript for detailed analysis")
print("=" * 60)


HELLO! I'M LYUDMILA, YOUR MEDICAL ASSISTANT

I'll help gather information about how you're feeling
before your appointment with the doctor. Everything you tell me
will be passed on to your physician.
------------------------------------------------------------



👤 What's your name?  Alexander



🤖 Lyudmila: Nice to meet you, Alexander!

🤖 Lyudmila: Please tell me what's bothering you?


👤 You:  I feel constantly weak.



🤖 Lyudmila: Anything else?


...


🤖 Lyudmila: I understand. So you're feeling sad because of the loss of a loved one. The grief and sense of loneliness likely contribute to these feelings. And is there anything specific that makes your mood worse? Is it certain thoughts, memories, or situations that trigger sadness?

And what helps improve your mood? Are there any activities, people, or things you find comforting or uplifting?



👤 You:  Thoughts about the past are depressing, and I still haven't let go of that situation. Distracting myself with video games helps.



🤖 Lyudmila: Okay, so dwelling on the past contributes to your sadness, and video games provide temporary relief. Is there anything else you do to cope with these feelings? Anything at all? Are there any coping mechanisms that help you manage these negative emotions? Also, have you tried any self-treatment for this weakness or bad mood?



👤 You:  No, but I decided to seek help from a doctor and am now having a preliminary conversation with you.



🤖 Lyudmila: Okay, thank you for sharing all of this information. It helps me understand your situation better. Now that we've covered the main complaints, let's summarize:

*   **Constant Weakness:** Develops gradually over years, worsened by bad mood.
*   **Difficulty Starting Things:** Related to lack of motivation and potentially depression, exacerbated by sadness and emptiness.
*   **Bad Mood/Depression:** Triggered by grief from the loss of a loved one, with thoughts about the past contributing to negativity; video games provide temporary relief.

I've gathered all the necessary information and will pass it to your doctor. Thank you for your cooperation.


✅ Report saved to file: report_Alexander_20260223_0914.txt

Report includes:
• Concise summary for quick review
• Full transcript for detailed analysis
